In [1]:
import os

os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"] = "0"

import tensorflow as tf

config = tf.ConfigProto()
config.gpu_options.allow_growth = True
sess = tf.Session(config=config)

import sys

if not '../' in sys.path: sys.path.append('../')

import pandas as pd

from utils import data_utils
from model_config import config
from ved_varAttn import VarSeq2SeqVarAttnModel

Using TensorFlow backend.


In [2]:
if config['experiment'] == 'qgen':
    print('[INFO] Preparing data for experiment: {}'.format(config['experiment']))
    train_data = pd.read_csv(config['data_dir'] + 'df_qgen_train.csv')
    val_data = pd.read_csv(config['data_dir'] + 'df_qgen_val.csv')
    test_data = pd.read_csv(config['data_dir'] + 'df_qgen_test.csv')
    input_sentences = pd.concat([train_data['answer'], val_data['answer'], test_data['answer']])
    output_sentences = pd.concat([train_data['question'], val_data['question'], test_data['question']])
    true_test = test_data['question']
    input_test = test_data['answer']
    filters = '!"#$%&()*+,./:;<=>?@[\\]^`{|}~\t\n'
    w2v_path = config['w2v_dir'] + 'w2vmodel_qgen.pkl'
    
elif config['experiment'] == 'dialogue':
    train_data = pd.read_csv(config['data_dir'] + 'df_dialogue_train.csv')
    val_data = pd.read_csv(config['data_dir'] + 'df_dialogue_val.csv')
    test_data = pd.read_csv(config['data_dir'] + 'df_dialogue_test.csv')
    input_sentences = pd.concat([train_data['line'], val_data['line'], test_data['line']])
    output_sentences = pd.concat([train_data['reply'], val_data['reply'], test_data['reply']])
    true_test = test_data['reply']
    input_test = test_data['line']
    filters = '!"#$%&()*+/:;<=>@[\\]^`{|}~\t\n'
    w2v_path = config['w2v_dir'] + 'w2vmodel_dialogue.pkl'

elif config['experiment'] == 'arc':
        train_data = pd.read_csv(config['data_dir'] + 'df_arc_train.csv')
        val_data = pd.read_csv(config['data_dir'] + 'df_arc_val.csv')
        test_data = pd.read_csv(config['data_dir'] + 'df_arc_test.csv')
        input_sentences = pd.concat([train_data['ProductSent'],
                                    val_data['ProductSent'],
                                    test_data['ProductSent']])
        output_sentences = pd.concat([train_data['ProductPhrase'],
                                      val_data['ProductPhrase'],
                                      test_data['ProductPhrase']])
        true_test = test_data['ProductPhrase']
        input_test = test_data['ProductSent']
        filters = '!"#$%&()*+/:;<=>@[\\]^`{|}~\t\n'
        w2v_path = config['w2v_dir'] + 'w2vmodel_arc.pkl'

else:
    print('Invalid experiment name specified!')
    

In [3]:
print('[INFO] Tokenizing input and output sequences')
x, input_word_index = data_utils.tokenize_sequence(input_sentences, 
                                                                      filters, 
                                                                      config['encoder_num_tokens'], 
                                                                      config['encoder_vocab'])

y, output_word_index = data_utils.tokenize_sequence(output_sentences, 
                                                                        filters, 
                                                                        config['decoder_num_tokens'], 
                                                                        config['decoder_vocab'])

print('[INFO] Split data into train-validation-test sets')
x_train, y_train, x_val, y_val, x_test, y_test = data_utils.create_data_split(x, 
                                                                              y, 
                                                                              config['experiment'])

encoder_embeddings_matrix = data_utils.create_embedding_matrix(input_word_index, 
                                                               config['embedding_size'], 
                                                               w2v_path)

decoder_embeddings_matrix = data_utils.create_embedding_matrix(output_word_index, 
                                                               config['embedding_size'], 
                                                               w2v_path)

# Re-calculate the vocab size based on the word_idx dictionary
config['encoder_vocab'] = len(input_word_index)
config['decoder_vocab'] = len(output_word_index)

[INFO] Tokenizing input and output sequences
[INFO] Split data into train-validation-test sets


In [4]:
model = VarSeq2SeqVarAttnModel(config, 
                               encoder_embeddings_matrix, 
                               decoder_embeddings_matrix, 
                               input_word_index, 
                               output_word_index)

[INFO] Building Model ...
Instructions for updating:
seq_dim is deprecated, use seq_axis instead
Instructions for updating:
batch_dim is deprecated, use batch_axis instead
Instructions for updating:
keep_dims is deprecated, use keepdims instead


In [5]:
if config['load_checkpoint'] != 0: 
    checkpoint = config['model_checkpoint_dir'] + str(config['load_checkpoint']) + '.ckpt'
else:
    checkpoint = tf.train.get_checkpoint_state(os.path.dirname('models/checkpoint')).model_checkpoint_path

preds = model.predict(checkpoint, 
                      x_test, 
                      y_test, 
                      true_test, 
                      )

INFO:tensorflow:Restoring parameters from models/var-seq2seq-var-attn-9.ckpt
BLEU 1 to 4 : 89.02 | 79.34 | 70.22 | 50.91


In [6]:
count = 100
model.show_output_sentences(preds[:count], 
                            y_test[:count], 
                            input_test[:count], 
                            true_test[:count], 
                            )

Input:      i have always used these for my crvs .
Actual:     use for crv
Generated: use for crv

Input:      we were sad when these were replaced by plastic tumblers at costco  we like to use them for hot liquids .
Actual:     use for hot liquid
Generated: use for liquid

Input:      for this reason  i would not use it inside to do a large bird - we placed it outside the garage on the concrete .
Actual:     use for reason
Generated: use for reason

Input:      no problems after using the toro for over 2 hours .
Actual:     use toro for hour
Generated: use toro for hour

Input:      used this product holistically for my terminally ill dog  and i loved it .
Actual:     use product for ill dog
Generated: use product for dog

Input:      surefire really made it so that it can be used for tactical purpose .
Actual:     use for purpose
Generated: use for purpose

Input:      my husband uses this for light tilling  especially for our vegetable garden .
Actual:     use for vegetable garden
G

In [7]:
model.get_diversity_metrics(checkpoint, x_test, y_test)

INFO:tensorflow:Restoring parameters from models/var-seq2seq-var-attn-9.ckpt


100%|██████████| 3/3 [11:24<00:00, 229.09s/it]

Entropy = 1.235 | Distinct-1 = 0.102 | Distinct-2 = 0.106
